This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

## Introduction to generative adversarial networks

### A schematic GAN implementation

### A bag of tricks

### Getting our hands on the CelebA dataset

**Getting the CelebA data**

In [ ]:
!mkdir celeba_gan # creating a directory for the dataset
!gdown --id 1O7m1010EJjLE5QxLZiM9Fpjs7Oj6e684 -O celeba_gan/data.zip # downloading the dataset
!unzip -qq celeba_gan/data.zip -d celeba_gan # unzipping the dataset

**Creating a dataset from a directory of images**

In [ ]:
from tensorflow import keras # importing the necessary libraries
dataset = keras.utils.image_dataset_from_directory( # loading the dataset
    "celeba_gan", # specifying the directory of the dataset 
    label_mode=None, # not using labels
    image_size=(64, 64), # resizing the images to 64x64 pixels
    batch_size=32, # setting the batch size
    smart_resize=True) # resizing the images to the specified size

**Rescaling the images**

In [ ]:
dataset = dataset.map(lambda x: x / 255.) # normalizing the images to the range [0, 1] 

**Displaying the first image**

In [ ]:
import matplotlib.pyplot as plt # importing the necessary libraries
for x in dataset: # iterating through the dataset
    plt.axis("off") # turning off the axis
    plt.imshow((x.numpy() * 255).astype("int32")[0]) # displaying the images in the dataset 
    break # breaking the loop after the first batch of images

### The discriminator

**The GAN discriminator network**

In [ ]:
from tensorflow.keras import layers # importing the necessary libraries

discriminator = keras.Sequential( # creating the discriminator model
    [
        keras.Input(shape=(64, 64, 3)), # specifying the input shape
        layers.Conv2D(64, kernel_size=4, strides=2, padding="same"), # adding the first convolutional layer
        layers.LeakyReLU(alpha=0.2), # adding the first activation layer
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"), # adding the second convolutional layer
        layers.LeakyReLU(alpha=0.2), # adding the second activation layer
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"), # adding the third convolutional layer
        layers.LeakyReLU(alpha=0.2), # adding the third activation layer
        layers.Flatten(), # flattening the output of the convolutional layers to feed into the dense layers
        layers.Dropout(0.2), # adding a dropout layer to prevent overfitting 
        layers.Dense(1, activation="sigmoid"), # adding the output layer with the sigmoid activation function 
    ],
    name="discriminator", # naming the model
)

In [ ]:
discriminator.summary() # displaying the architecture of the discriminator model

### The generator

**GAN generator network**

In [ ]:
latent_dim = 128 # setting the dimensionality of the latent space

generator = keras.Sequential( # creating the generator model
    [
        keras.Input(shape=(latent_dim,)), # specifying the input shape
        layers.Dense(8 * 8 * 128), # adding the first dense layer
        layers.Reshape((8, 8, 128)), # reshaping the output of the dense layer to feed into the convolutional layers
        layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"), # adding the first convolutional layer
        layers.LeakyReLU(alpha=0.2), # adding the first activation layer
        layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"), # adding the second convolutional layer
        layers.LeakyReLU(alpha=0.2), # adding the second activation layer
        layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"), # adding the third convolutional layer
        layers.LeakyReLU(alpha=0.2), # adding the third activation layer
        layers.Conv2D(3, kernel_size=5, padding="same", activation="sigmoid"), # adding the output layer with the sigmoid activation function
    ],
    name="generator", # naming the model
)

In [ ]:
generator.summary() # displaying the architecture of the generator model

### The adversarial network

**The GAN `Model`**

In [ ]:
import tensorflow as tf # importing the tensorflow library 
class GAN(keras.Model): # creating the GAN class
    def __init__(self, discriminator, generator, latent_dim): # defining the constructor
        super().__init__() # calling the constructor of the parent class
        self.discriminator = discriminator # initializing the discriminator model
        self.generator = generator # initializing the generator model
        self.latent_dim = latent_dim # initializing the dimensionality of the latent space
        self.d_loss_metric = keras.metrics.Mean(name="d_loss") # initializing the discriminator loss metric
        self.g_loss_metric = keras.metrics.Mean(name="g_loss") # initializing the generator loss metric

    def compile(self, d_optimizer, g_optimizer, loss_fn): # defining the compile method
        super(GAN, self).compile() # calling the compile method of the parent class
        self.d_optimizer = d_optimizer # initializing the discriminator optimizer
        self.g_optimizer = g_optimizer # initializing the generator optimizer
        self.loss_fn = loss_fn # initializing the loss function

    @property # defining the property decorator
    def metrics(self): # defining the metrics method
        return [self.d_loss_metric, self.g_loss_metric] # returning the discriminator and generator loss metrics

    def train_step(self, real_images): # defining the train_step method
        batch_size = tf.shape(real_images)[0] # getting the batch size
        random_latent_vectors = tf.random.normal( # generating random latent vectors
            shape=(batch_size, self.latent_dim)) # specifying the shape of the latent vectors
        generated_images = self.generator(random_latent_vectors) # generating images using the generator model
        combined_images = tf.concat([generated_images, real_images], axis=0) # combining the generated and real images
        labels = tf.concat( # creating the labels for the discriminator
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], # specifying the labels for the generated and real images
            axis=0 # concatenating the labels along the rows
        )
        labels += 0.05 * tf.random.uniform(tf.shape(labels)) # adding noise to the labels

        with tf.GradientTape() as tape: # defining the gradient tape
            predictions = self.discriminator(combined_images) # getting the predictions of the discriminator
            d_loss = self.loss_fn(labels, predictions) # calculating the discriminator loss
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights) # calculating the gradients
        self.d_optimizer.apply_gradients( # applying the gradients to the discriminator model
            zip(grads, self.discriminator.trainable_weights) # zipping the gradients and trainable weights
        )

        random_latent_vectors = tf.random.normal( # generating new random latent vectors
            shape=(batch_size, self.latent_dim)) # specifying the shape of the latent vectors

        misleading_labels = tf.zeros((batch_size, 1)) # creating misleading labels for the generator

        with tf.GradientTape() as tape: # defining the gradient tape
            predictions = self.discriminator( # getting the predictions of the discriminator
                self.generator(random_latent_vectors)) # generating images using the generator model
            g_loss = self.loss_fn(misleading_labels, predictions) # calculating the generator loss
        grads = tape.gradient(g_loss, self.generator.trainable_weights) # calculating the gradients
        self.g_optimizer.apply_gradients( # applying the gradients to the generator model
            zip(grads, self.generator.trainable_weights)) # zipping the gradients and trainable weights

        self.d_loss_metric.update_state(d_loss) # updating the discriminator loss metric
        self.g_loss_metric.update_state(g_loss) # updating the generator loss metric
        return {"d_loss": self.d_loss_metric.result(), # returning the discriminator and generator loss metrics for d_loss
                "g_loss": self.g_loss_metric.result()} # returning the discriminator and generator loss metrics for g_loss

**A callback that samples generated images during training**

In [ ]:
class GANMonitor(keras.callbacks.Callback): # creating the GANMonitor class
    def __init__(self, num_img=3, latent_dim=128): # defining the constructor
        self.num_img = num_img # initializing the number of images to generate
        self.latent_dim = latent_dim # initializing the dimensionality of the latent space

    def on_epoch_end(self, epoch, logs=None): # defining the on_epoch_end method
        random_latent_vectors = tf.random.normal(shape=(self.num_img, self.latent_dim)) # generating random latent vectors
        generated_images = self.model.generator(random_latent_vectors) # generating images using the generator model
        generated_images *= 255 # rescaling the images to the range [0, 255]
        generated_images.numpy() # converting the images to a numpy array
        for i in range(self.num_img): # iterating through the generated images
            img = keras.utils.array_to_img(generated_images[i]) # converting the image to a PIL image
            img.save(f"generated_img_{epoch:03d}_{i}.png") # saving the image

**Compiling and training the GAN**

In [ ]:
epochs = 100 # setting the number of epochs

gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim) # creating the GAN model
gan.compile( # compiling the GAN model
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0001), # specifying the discriminator optimizer
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0001), # specifying the generator optimizer
    loss_fn=keras.losses.BinaryCrossentropy(), # specifying the loss function
)

gan.fit( # training the GAN model
    dataset, epochs=epochs, callbacks=[GANMonitor(num_img=10, latent_dim=latent_dim)] # specifying the dataset, number of epochs, and callbacks 
)

### Wrapping up

## Summary